## Downloading Email Headers

In [1]:
username = "krishnaraj.kpt@outlook.com"
password = "BBQtJTSs8uQh57aw"
imap_server = "outlook.office365.com"

In [5]:
import imaplib
import email
from email.header import decode_header
from email.parser import BytesParser
import json

Creating Imap Object

In [6]:
# create an IMAP4 class with SSL
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)
status, messages = imap.select("INBOX")

# number of top emails to fetch
N = 15

# total number of emails
messages = int(messages[0])
print("The Total Number of Messages in your account are: ", messages)

The Total Number of Messages in your account are:  1017


Downloading Headers

In [7]:
# create a list to store the emails
emails = []

# fetch the top N email headers
for i in range(messages - N, messages + 1):
    # fetch the email header
    result, data = imap.fetch(str(i + 1), "(RFC822.HEADER)")
    if result == "OK":
        # parse the email header
        email_parser = BytesParser()
        email_header = email_parser.parsebytes(data[0][1])

        # create a dictionary to store the email header
        email = {
            "subject": email_header["Subject"],
            "headers": [],
        }

        # add the email headers to the dictionary
        for header in email_header.items():
            email["headers"].append({
                "header": header[0],
                "value": header[1],
            })

        # add the email to the list of emails
        emails.append(email)

# close the IMAP connection
imap.close()

# write the emails to a JSON file
with open("emails.json", "w") as f:
    json.dump(emails, f, indent=4)

Converting the json file to a python dictionary

In [92]:
import json
with open("emails.json", "r") as f:
    emails = json.load(f)

Looking at Header keys of one of the emails

In [93]:
header_names = [i['header'] for i in emails[1]["headers"]]

Focusing on 8 Test Emails

In [94]:
emails = emails[2:20]

Looking at their Subjects

In [95]:
[i['subject'] for i in emails]

['Sending from phone gmail app',
 'Self mail from web outlook client',
 'Self test mail from gmail web client',
 'self test mail from mit gmail client',
 'From movile outlook app',
 'Mail sent from phone',
 "Mother's phone Gmail app",
 'Sending from browser with vpn connection on',
 'Sending from browser with vpn connection on',
 'Sending from browser using mobile hotspot',
 '[weekly] report for 2023-10-23 until 2023-10-29',
 'Spell Casting, Ghostly Reveals, and a Halloween Party',
 '[GitHub] Your Dependabot alerts for the week of Oct 24 - Oct 31']

Analyse each header and find out what it means

In [96]:
print("Total number of headers in provided email: ", len(header_names))

Total number of headers in provided email:  57


In [97]:
# print header names with their index
for index, header in enumerate(header_names):
    print(index, header)

0 MIME-Version
1 Received
2 Received
3 Received
4 Authentication-Results
5 Received-SPF
6 Received
7 X-IncomingTopHeaderMarker
8 Received
9 DKIM-Signature
10 X-Google-DKIM-Signature
11 X-Gm-Message-State
12 X-Google-Smtp-Source
13 X-Received
14 From
15 Date
16 Message-ID
17 Subject
18 To
19 Content-Type
20 X-IncomingHeaderCount
21 Return-Path
22 X-MS-Exchange-Organization-ExpirationStartTime
23 X-MS-Exchange-Organization-ExpirationStartTimeReason
24 X-MS-Exchange-Organization-ExpirationInterval
25 X-MS-Exchange-Organization-ExpirationIntervalReason
26 X-MS-Exchange-Organization-Network-Message-Id
27 X-EOPAttributedMessage
28 X-EOPTenantAttributedMessage
29 X-MS-Exchange-Organization-MessageDirectionality
30 X-MS-PublicTrafficType
31 X-MS-TrafficTypeDiagnostic
32 X-MS-Exchange-Organization-AuthSource
33 X-MS-Exchange-Organization-AuthAs
34 X-MS-UserLastLogonTime
35 X-MS-Office365-Filtering-Correlation-Id
36 X-MS-Exchange-EOPDirect
37 X-Sender-IP
38 X-SID-PRA
39 X-SID-Result
40 X-MS-Exch

### MIME-Version

MIME-Version indicates the email's message format. It's important in investigations to understand how the message is structured and if it includes multimedia or attachments.

In [73]:
emails[1]['headers'][0]['header'], emails[1]['headers'][0]['value']

('MIME-Version', '1.0')

### Received

Received headers track the path of the email through various servers. This is crucial for tracing the email's journey, identifying potential anomalies, or investigating its source.


In [74]:
emails[1]['headers'][1]['header'], emails[1]['headers'][1]['value']

('Received',
 'from CY8PR17MB6329.namprd17.prod.outlook.com (2603:10b6:930:9c::12)\r\n by DS7PR17MB6730.namprd17.prod.outlook.com with HTTPS; Sun, 29 Oct 2023\r\n 17:38:15 +0000')

### ARC-Seal

ARC (Authenticated Received Chain) headers help verify the authenticity of email forwarding. ARC-Seal ensures the integrity of email headers, reducing the risk of spoofing.


In [75]:
emails[1]['headers'][2]['header'], emails[1]['headers'][2]['value']

('Received',
 'from GVX0EPF000013EC.SWEP280.PROD.OUTLOOK.COM (2603:10a6:144:1::19)\r\n by CY8PR17MB6329.namprd17.prod.outlook.com (2603:10b6:930:9c::12) with\r\n Microsoft SMTP Server (version=TLS1_2,\r\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.6954.8; Sun, 29 Oct\r\n 2023 17:38:14 +0000')

### ARC-Message-Signature

ARC-Message-Signature is part of ARC headers and provides cryptographic assurance of email header integrity, aiding in detecting email tampering.


In [76]:
emails[1]['headers'][3]['header'], emails[1]['headers'][3]['value']

('Received',
 'from HE1EUR04FT029.eop-eur04.prod.protection.outlook.com\r\n (2a01:111:f400:7e0d::208) by GVX0EPF000013EC.outlook.office365.com\r\n (2603:1026:900:2::3) with Microsoft SMTP Server (version=TLS1_2,\r\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.6792.35 via Frontend\r\n Transport; Sun, 29 Oct 2023 17:38:13 +0000')

### ARC-Authentication-Results

These headers indicate the email's authentication status. Investigators can use this to assess the email's legitimacy and potential for phishing.


In [77]:
emails[1]['headers'][4]['header'], emails[1]['headers'][4]['value']

('Authentication-Results',
 'spf=pass (sender IP is 209.85.160.175)\r\n smtp.mailfrom=gmail.com; dkim=pass (signature was verified)\r\n header.d=gmail.com;dmarc=pass action=none header.from=gmail.com;compauth=pass\r\n reason=100')

### DKIM-Signature

DKIM (DomainKeys Identified Mail) verifies that the email content hasn't been altered in transit. It's crucial for email integrity checks and source verification.


In [78]:
emails[1]['headers'][14]['header'], emails[1]['headers'][14]['value']

('From', 'VEDANG KHARE <vedangkhare28@gmail.com>')

### From

The "From" header shows the sender's email address. It's essential for identifying the sender, although it can be spoofed.


In [79]:
emails[1]['headers'][17]['header'], emails[1]['headers'][17]['value']

('Subject', 'Mail sent from phone')

### To

The "To" header reveals the email's recipient, which is significant for understanding the email's target and potential threat actors.


In [80]:
emails[1]['headers'][18]['header'], emails[1]['headers'][18]['value']

('To', 'krishnaraj.kpt@outlook.com')

### Subject

The subject line provides insight into the email's content, which is crucial for assessing the email's purpose and relevance to an investigation.


In [81]:
emails[1]['headers'][19]['header'], emails[1]['headers'][19]['value']

('Content-Type',
 'multipart/alternative; boundary="000000000000c264ca0608de62f0"')

### Date

The date header shows when the email was sent. It's valuable for establishing timelines and correlations with other events.


In [82]:
emails[1]['headers'][22]['header'], emails[1]['headers'][22]['value']

('X-MS-Exchange-Organization-ExpirationStartTime',
 '29 Oct 2023 17:38:13.5337\r\n (UTC)')

### Message-ID

The Message-ID is unique to each email and can be used for tracking and associating related messages in an investigation.


In [83]:
emails[1]['headers'][23]['header'], emails[1]['headers'][23]['value']

('X-MS-Exchange-Organization-ExpirationStartTimeReason', 'OriginalSubmit')

### Content-Type

Content-Type specifies the format of the email content. It helps investigators interpret the email's structure and potential for malicious attachments.


In [84]:
emails[1]['headers'][36]['header'], emails[1]['headers'][36]['value']

('X-MS-Exchange-EOPDirect', 'true')

### Return-Path

Return-Path indicates where undeliverable emails should be sent. It can assist in identifying email redirection or bouncing patterns.


In [85]:
emails[1]['headers'][39]['header'], emails[1]['headers'][39]['value']

('X-SID-Result', 'PASS')

### X-Sender-IP

This header contains the IP address of the sender, which is essential for tracking the origin of the email and potential geolocation.


In [86]:
emails[1]['headers'][56]['header'], emails[1]['headers'][56]['value']

('X-Microsoft-Antispam-Message-Info',
 '\r\n\t=?utf-8?B?RUtYZHI2dzBrVlNHOTBnNTZYSHVydXFLNzEvZnd1eWMyZEU1QloyVE5iVGl6?=\r\n =?utf-8?B?RWxpWHJKTHcwTlcvVm5mdjY2ZTgvT252cHZPektaSjJ3U3FVanZ5dDV6T214?=\r\n =?utf-8?B?d0cwaWFnakpNUEJDS3BPQzhYZDZwL2NQMGhWckRtb1JpQ05GM3JuRFA0MVps?=\r\n =?utf-8?B?ejBBMU8yaXgrNWhvZmRHTHNtcWlLR1JYR3NsZjdZYjFBUHRDR1JqbU9sK2hx?=\r\n =?utf-8?B?aXFFejZqUGEvYWQzdVRvc09zMGhBcEFiYWFnOUhiMUx4MjJtWlZ0QU04aDc3?=\r\n =?utf-8?B?blgwWVNZQ0FMcCtxV2JmVFJZYVc3WHhyUXl3WFBjQWltOFFzZVp6aFNkbXRC?=\r\n =?utf-8?B?ekk2TGcrK0RHenVzMlBSaW9la1ZEV3F5U21JUk9Xd1RkUGxLZE5ZUGlsY2kv?=\r\n =?utf-8?B?MnRZdDVpTnhDeEsreWdWSjVITC91Wnd6bWtvaWM0Uk5QL1FIMTc0WlVUcUtZ?=\r\n =?utf-8?B?MlBJRDNtaHdUMzVpUG9mVWtGR1BTVDRmR20vMWZEZDZnTkdTbHdudU8vUWxY?=\r\n =?utf-8?B?M2dGMC9jZ3VHY21wUjIrRXNHMS9EVFd5Uno5UnNIZXprc3E2L0JMMWRHaXRs?=\r\n =?utf-8?B?OFdEblNOaDF0cEFlQkhkanppZVNKTUFlRzdzUS9wYkNRdTdpd0NIZlBNQy9M?=\r\n =?utf-8?B?aC8xUlFhWmlWTkhNNHdFNEVVVFpoeG9MRTRsWHVmOXhxcXhVbkcvVHhxQ3Uw?=\r\n =?utf-8?B?ZkJkMnFLcmxxckx3czVY

### X-MS-Exchange-Transport-EndToEndLatency

End-to-end latency is crucial for assessing the email's delivery speed, which might reveal anomalies or delays in transit.


In [87]:
emails[1]['headers'][71]['header'], emails[1]['headers'][71]['value']

IndexError: list index out of range

### X-Microsoft-Antispam-Mailbox-Delivery

This header provides information about the email's delivery and its classification as spam or not, aiding in filtering and threat analysis.


In [88]:
emails[1]['headers'][73]['header'], emails[1]['headers'][73]['value']

IndexError: list index out of range

### X-Message-Info and X-Message-Delivery

These headers contain miscellaneous information about the email's handling, which can be valuable for tracking and understanding the email's journey.

In [89]:
emails[1]['headers'][75]['header'], emails[1]['headers'][75]['value']

IndexError: list index out of range

Defining a function to get information about an ip address for sender ip analysis

In [90]:
import requests

def get_ip_information(ip_address):
    def get_ip_location():
        # Make a GET request to ipinfo.io with the IP address
        url = f"https://ipinfo.io/{ip_address}/json"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            return None

    location_info = get_ip_location()
    # print(location_info)
    if location_info:
        # Print the location information
        print(f"IP Address: {location_info['ip']}")
        print(f"Hostname: {location_info['hostname']}")
        print(f"City: {location_info['city']}")
        # print(f"Region: {location_info['region']}")
        print(f"Country: {location_info['country']}")
        print(f"Location: {location_info['loc']}")
        # print(f"Organization: {location_info['org']}")
        print(f"Timezone: {location_info['timezone']}")
    else:
        print("Unable to retrieve location information for the IP address.")


Analysing Sender IP Address for all emails

In [98]:
for email in emails:
    for header in email['headers']:
        if header['header'] == 'X-Sender-IP':
            print("Subject of Email: ", email['subject'])
            print("IP Address Information:")
            get_ip_information(header['value'])

            print()

Subject of Email:  Sending from phone gmail app
IP Address Information:
IP Address: 209.85.218.48
Hostname: mail-ej1-f48.google.com
City: Oudeschip
Country: NL
Location: 53.4300,6.8264
Timezone: Europe/Amsterdam

Subject of Email:  Self mail from web outlook client
IP Address Information:
IP Address: 40.92.20.10
Hostname: mail-bn8nam11olkn2010.outbound.protection.outlook.com
City: Boydton
Country: US
Location: 36.6676,-78.3875
Timezone: America/New_York

Subject of Email:  Self test mail from gmail web client
IP Address Information:
IP Address: 209.85.167.178
Hostname: mail-oi1-f178.google.com
City: Tulsa
Country: US
Location: 36.1540,-95.9928
Timezone: America/Chicago

Subject of Email:  self test mail from mit gmail client
IP Address Information:
IP Address: 209.85.219.47
Hostname: mail-qv1-f47.google.com
City: Raleigh
Country: US
Location: 35.7721,-78.6386
Timezone: America/New_York

Subject of Email:  From movile outlook app
IP Address Information:
IP Address: 40.92.242.29
Hostname